In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import json
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import ensemble
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid", font_scale=1.5)
%matplotlib inline

In [8]:
listings = pd.read_csv('listings.csv',sep=',',header=0)

/Users/DiegoR/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
listings.head(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,9147025,https://www.airbnb.com/rooms/9147025,20160601225828,2016-06-02,Cozy City Island Cottage,City Island is a small community at the edge o...,This cozy little cottage is an open floor plan...,City Island is a small community at the edge o...,none,City Island is famous for it's seafood restaur...,...,9,f,NaN,NaN,f,moderate,f,f,1,1.69
1,6627449,https://www.airbnb.com/rooms/6627449,20160601225828,2016-06-02,Large 1 BDRM in Great location,This ground floor apartment is light and airy ...,"We are close to fishing, boating, biking, hors...",This ground floor apartment is light and airy ...,none,City Island is a unique and a hidden gem of Ne...,...,9,f,NaN,NaN,f,flexible,f,f,1,0.81
2,11675715,https://www.airbnb.com/rooms/11675715,20160601225828,2016-06-02,Cozy 1 BR Basement Apartment,"City Island is an active, year-round nautical ...",This very cozy apartment with features a queen...,"City Island is an active, year-round nautical ...",none,City Island is a small island and a neighborho...,...,10,f,NaN,NaN,t,flexible,f,f,1,4.46
3,8053533,https://www.airbnb.com/rooms/8053533,20160601225828,2016-06-02,Spacious Duplex Apartment,"Duplex apartment with fireplace, two balconies...",NaN,"Duplex apartment with fireplace, two balconies...",none,City Island is one of the gems of NYC. We are...,...,10,f,NaN,NaN,t,flexible,f,f,1,0.63
4,1886820,https://www.airbnb.com/rooms/1886820,20160601225828,2016-06-02,Quaint City Island Community.,Quiet island boating town on Long Island Soun...,"Master bed with queen bed, full bath and offi...",Quiet island boating town on Long Island Soun...,none,Small New England type town in the middle of ...,...,NaN,f,NaN,NaN,f,strict,f,f,1,NaN
5,5557381,https://www.airbnb.com/rooms/5557381,20160601225828,2016-06-02,Quaint City Island Home,"Located in an old sea-shanty town, our home ha...",You won't find a place so close to the city (N...,"Located in an old sea-shanty town, our home ha...",none,"City Island is unique in two ways. First, you ...",...,10,f,NaN,NaN,t,moderate,f,f,1,4.56
6,7949480,https://www.airbnb.com/rooms/7949480,20160601225828,2016-06-02,City Island Sanctuary Sunny BR & BA,Come relax in our quiet guest room w/adjacent ...,On parle français et anglais! We have moved to...,Come relax in our quiet guest room w/adjacent ...,none,City Island is a unique sanctuary in New York ...,...,10,f,NaN,NaN,f,moderate,t,t,1,1.75
7,3943387,https://www.airbnb.com/rooms/3943387,20160601225828,2016-06-02,Cozy City Island Room/Private Bath,Private area on 3rd floor of quaint City Islan...,City Island is a small community at the edge o...,Private area on 3rd floor of quaint City Islan...,none,NaN,...,10,f,NaN,NaN,f,moderate,f,f,1,0.74
8,958444,https://www.airbnb.com/rooms/958444,20160601225828,2016-06-02,Great 1BD waterfront City Island NY,NaN,I am renting my lovely one bedroom apartment t...,I am renting my lovely one bedroom apartment t...,none,City island is small island with about 4.500 p...,...,10,f,NaN,NaN,f,strict,f,t,1,1.49
9,501568,https://www.airbnb.com/rooms/501568,20160601225828,2016-06-02,Affordable Private Room Homestay,A cool and comfortable private room has a true...,The Bronx is burning hot...with great things t...,A cool and comfortable private room has a true...,none,"Nearby attractions: City Island, NY Botancial ...",...,10,f,NaN,NaN,f,strict,t,t,6,0.50


In [4]:
listings.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,36608.000000,3.660800e+04,36608.000000,36602.000000,36602.000000,36608.000000,36608.000000,36608.000000,36392.00000,36545.000000,...,26802.000000,26679.000000,26726.000000,26682.000000,26736.000000,26693.000000,26692.000000,0,36608.000000,27510.000000
mean,6909004.696924,2.016060e+13,20001580.146088,2.539479,2.539479,40.731358,-73.958610,2.812363,1.11971,1.137584,...,92.414036,9.462461,9.118499,9.636309,9.699693,9.343985,9.243181,NaN,1.860577,1.296519
std,3998510.857789,1.569943e+01,19602699.671704,18.244699,18.244699,0.052257,0.038339,1.779114,0.37900,0.680649,...,8.836644,0.876772,1.159202,0.772031,0.712438,0.905913,0.919903,NaN,3.013697,1.295557
min,2515.000000,2.016060e+13,283.000000,1.000000,1.000000,40.499794,-74.240843,1.000000,0.00000,0.000000,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,1.000000,0.010000
25%,3607738.750000,2.016060e+13,3779913.750000,1.000000,1.000000,40.693748,-73.985265,2.000000,1.00000,1.000000,...,90.000000,9.000000,9.000000,9.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.340000
50%,7290362.500000,2.016060e+13,12457092.000000,1.000000,1.000000,40.725278,-73.959370,2.000000,1.00000,1.000000,...,94.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.000000,NaN,1.000000,0.880000
75%,10034519.000000,2.016060e+13,32784479.750000,2.000000,2.000000,40.764030,-73.942077,4.000000,1.00000,1.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,1.900000
max,13318221.000000,2.016060e+13,75304227.000000,685.000000,685.000000,40.912316,-73.712116,16.000000,8.00000,10.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,40.000000,14.260000


In [5]:
listings.dtypes

id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
summary                              object
space                                object
description                          object
experiences_offered                  object
neighborhood_overview                object
notes                                object
transit                              object
access                               object
interaction                          object
house_rules                          object
thumbnail_url                        object
medium_url                           object
picture_url                          object
xl_picture_url                       object
host_id                               int64
host_url                             object
host_name                            object
host_since                      

In [6]:
# calendar = pd.read_csv('calendar.csv',sep=',',header=0)

In [7]:
# calendar.tail(10)

In [8]:
# calendar.date.min()

In [9]:
# calendar.date.max()

In [10]:
# listings.columns = listings.columns[] 

SyntaxError: invalid syntax (<ipython-input-10-aba6424ea519>, line 1)

In [11]:
listings[['availability_30', 'availability_60', 'availability_90', 'availability_365']].describe()

,availability_30,availability_60,availability_90,availability_365
count,36608.000000,36608.000000,36608.000000,36608.000000
mean,7.958588,19.694957,33.113855,162.407998
std,10.261638,21.438730,32.825425,144.046572
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,4.000000
50%,3.000000,11.000000,23.000000,152.000000
75%,14.000000,37.000000,63.000000,313.000000
max,30.000000,60.000000,90.000000,365.000000


In [16]:
len(listings[listings['availability_365']==365])

2149

In [17]:
len(listings)

36608

In [18]:
# Set label as class 1 (hot property) if availability is low
listings['y']=listings['availability_30']<10

In [19]:
listings[['availability_30','y']].head()

,availability_30,y
0,1,True
1,25,False
2,1,True
3,0,True
4,30,False


In [20]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,y
0,9147025,https://www.airbnb.com/rooms/9147025,20160601225828,2016-06-02,Cozy City Island Cottage,City Island is a small community at the edge o...,This cozy little cottage is an open floor plan...,City Island is a small community at the edge o...,none,City Island is famous for it's seafood restaur...,...,f,NaN,NaN,f,moderate,f,f,1,1.69,True
1,6627449,https://www.airbnb.com/rooms/6627449,20160601225828,2016-06-02,Large 1 BDRM in Great location,This ground floor apartment is light and airy ...,"We are close to fishing, boating, biking, hors...",This ground floor apartment is light and airy ...,none,City Island is a unique and a hidden gem of Ne...,...,f,NaN,NaN,f,flexible,f,f,1,0.81,False
2,11675715,https://www.airbnb.com/rooms/11675715,20160601225828,2016-06-02,Cozy 1 BR Basement Apartment,"City Island is an active, year-round nautical ...",This very cozy apartment with features a queen...,"City Island is an active, year-round nautical ...",none,City Island is a small island and a neighborho...,...,f,NaN,NaN,t,flexible,f,f,1,4.46,True
3,8053533,https://www.airbnb.com/rooms/8053533,20160601225828,2016-06-02,Spacious Duplex Apartment,"Duplex apartment with fireplace, two balconies...",NaN,"Duplex apartment with fireplace, two balconies...",none,City Island is one of the gems of NYC. We are...,...,f,NaN,NaN,t,flexible,f,f,1,0.63,True
4,1886820,https://www.airbnb.com/rooms/1886820,20160601225828,2016-06-02,Quaint City Island Community.,Quiet island boating town on Long Island Soun...,"Master bed with queen bed, full bath and offi...",Quiet island boating town on Long Island Soun...,none,Small New England type town in the middle of ...,...,f,NaN,NaN,f,strict,f,f,1,NaN,False


In [3]:
calendar = pd.read_csv('calendar.csv',sep=',',header=0)

In [4]:
calendar.describe()

,listing_id
count,13361920.000000
mean,6909004.696924
std,3998456.394514
min,2515.000000
25%,3607738.750000
50%,7290362.500000
75%,10034519.000000
max,13318221.000000


In [4]:
calendar.head(10).ascending()

NameError: name 'calendar' is not defined

In [5]:
calendar.columns

NameError: name 'calendar' is not defined

In [6]:
reviews = pd.read_csv('reviews.csv',sep=',',header=0)

In [16]:
reviews.describe

<bound method NDFrame.describe of         listing_id        date
0         10413399  2016-03-27
1         10413399  2016-04-05
2         10413399  2016-04-21
3         10413399  2016-04-29
4         10413399  2016-05-02
5         10413399  2016-05-10
6         10413399  2016-05-18
7         10413399  2016-05-22
8         12221331  2016-05-10
9          4796592  2015-04-23
10          447766  2014-08-21
11         6132757  2015-05-19
12         6132757  2015-07-17
13         6132757  2015-07-20
14         6132757  2015-08-07
15         6132757  2015-08-20
16         6132757  2015-08-25
17         6132757  2015-08-29
18         6132757  2015-09-13
19         6132757  2015-09-20
20         6132757  2015-09-21
21         6132757  2015-09-22
22         6132757  2015-09-26
23         6132757  2015-09-29
24         6132757  2015-10-05
25         6132757  2015-10-09
26         6132757  2015-10-15
27         6132757  2015-10-18
28         6132757  2015-10-28
29         6132757  2015-10-30
...  

In [17]:
reviews.columns

Index(['listing_id', 'date'], dtype='object')

In [20]:
neighbourhoods = pd.read_csv('neighbourhoods.csv',sep=',',header=0)

In [23]:
neighbourhoods.describe

<bound method NDFrame.describe of     neighbourhood_group      neighbourhood
0                 Bronx           Allerton
1                 Bronx         Baychester
2                 Bronx            Belmont
3                 Bronx          Bronxdale
4                 Bronx        Castle Hill
5                 Bronx        City Island
6                 Bronx  Claremont Village
7                 Bronx       Clason Point
8                 Bronx         Co-op City
9                 Bronx          Concourse
10                Bronx  Concourse Village
11                Bronx       Country Club
12                Bronx    East Morrisania
13                Bronx        Eastchester
14                Bronx           Edenwald
15                Bronx          Fieldston
16                Bronx            Fordham
17                Bronx         Highbridge
18                Bronx        Hunts Point
19                Bronx        Kingsbridge
20                Bronx           Longwood
21                Br

In [25]:
neighbourhoods.head(10)

,neighbourhood_group,neighbourhood
0,Bronx,Allerton
1,Bronx,Baychester
2,Bronx,Belmont
3,Bronx,Bronxdale
4,Bronx,Castle Hill
5,Bronx,City Island
6,Bronx,Claremont Village
7,Bronx,Clason Point
8,Bronx,Co-op City
9,Bronx,Concourse


In [51]:
calendar.head(200)

,listing_id,date,available,price
0,3646059,2016-06-02,f,NaN
1,3646059,2016-06-03,f,NaN
2,3646059,2016-06-04,f,NaN
3,3646059,2016-06-05,f,NaN
4,3646059,2016-06-06,f,NaN
5,3646059,2016-06-07,f,NaN
6,3646059,2016-06-08,f,NaN
7,3646059,2016-06-09,f,NaN
8,3646059,2016-06-10,t,$65.00
9,3646059,2016-06-11,f,NaN


In [35]:
calendar.date

0           2016-06-02
1           2016-06-03
2           2016-06-04
3           2016-06-05
4           2016-06-06
5           2016-06-07
6           2016-06-08
7           2016-06-09
8           2016-06-10
9           2016-06-11
10          2016-06-12
11          2016-06-13
12          2016-06-14
13          2016-06-15
14          2016-06-16
15          2016-06-17
16          2016-06-18
17          2016-06-19
18          2016-06-20
19          2016-06-21
20          2016-06-22
21          2016-06-23
22          2016-06-24
23          2016-06-25
24          2016-06-26
25          2016-06-27
26          2016-06-28
27          2016-06-29
28          2016-06-30
29          2016-07-01
               ...    
13361890    2017-05-03
13361891    2017-05-04
13361892    2017-05-05
13361893    2017-05-06
13361894    2017-05-07
13361895    2017-05-08
13361896    2017-05-09
13361897    2017-05-10
13361898    2017-05-11
13361899    2017-05-12
13361900    2017-05-13
13361901    2017-05-14
13361902   

In [44]:
X = listings.loc[0:10, :]

In [45]:
X.shape

(11, 95)

In [7]:
listings['has_availability']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
         ..
36578   NaN
36579   NaN
36580   NaN
36581   NaN
36582   NaN
36583   NaN
36584   NaN
36585   NaN
36586   NaN
36587   NaN
36588   NaN
36589   NaN
36590   NaN
36591   NaN
36592   NaN
36593   NaN
36594   NaN
36595   NaN
36596   NaN
36597   NaN
36598   NaN
36599   NaN
36600   NaN
36601   NaN
36602   NaN
36603   NaN
36604   NaN
36605   NaN
36606   NaN
36607   NaN
Name: has_availability, dtype: float64

In [9]:
# Drop missing values
listings = listings.dropna()

In [11]:
# Set features to use
existing_features = ['latitude', 'longitude', 'accommodates', 'price','availability_30','availability_60','availability_90','availability_365']
new_text_features = []
features = existing_features # + new_text_features

# Set target variable name
target = 'label'
#this is the complicated part

# Set X and y
X = listings[features]
y = listings[target]

# Create separate training and test sets
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=42)


# Instantiate model
clf = ensemble.RandomForestClassifier(n_estimators=200)

# Train model on training set
clf.fit(X_train, y_train)


# Evaluate accuracy of model on test set
print("Accuracy: %0.3f") % clf.score(X_test, y_test)

# Evaluate ROC AUC score of model on test set
print('ROC AUC: %0.3f') % metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])


# Plot feature importances
feature_names = X.columns
feature_importances = clf.feature_importances_
feature_dict = dict(zip(feature_names, feature_importances))

features_df = pd.DataFrame(feature_dict.items(), columns=['Features', 'Importance Score'])
features_df.sort_values('Importance Score', inplace=True, ascending=False)
sns.barplot(y='Features', x='Importance Score', data=features_df)

KeyError: 'label'

In [ ]:
'''will provide an updated file with data viz once I have spent more time this weekend with project.
work has been unbearable and I haven't been able to do as much as I had hoped, but will likely catchup this weekend.'''